<a href="https://colab.research.google.com/github/Uriel1201/HelloSQL2.0/blob/main/15_match_standings/query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##15. Team Standings.

**Write a query to return the scores of
each team in the teams table after all
matches displayed in the matches table.
The result should include team name and points,
and be ordered by decreasing points. In case of
a tie, order by alphabetized team name.**

In [ ]:
!pip install polars
import pandas as pd
import numpy as  np
import polars as pl
url1="https://raw.githubusercontent.com/Uriel1201/HelloSQL2.0/refs/heads/main/15_match_standings/data_matches.tsv"
url2="https://raw.githubusercontent.com/Uriel1201/HelloSQL2.0/refs/heads/main/15_match_standings/data_teams.tsv"

In [ ]:
pd_matches=pd.read_csv(url1,sep='\t')
pd_teams=pd.read_csv(url2,sep='\t')

In [ ]:
print(f'----PANDAS----\nmatches table:\n{pd_matches}')

----PANDAS----
matches table:
   MATCH_ID  HOST_TEAM  GUEST_TEAM  HOST_GOALS  GUEST_GOALS
0         1          1           2           3            0
1         2          2           3           2            4
2         3          3           4           4            3
3         4          4           5           1            1
4         5          5           6           2            1
5         6          6           1           1            2


In [ ]:
print(f'teams table:\n{pd_teams}')

teams table:
   TEAM_ID    TEAM_NAME
0        1     New York
1        2      Atlanta
2        3      Chicago
3        4      Toronto
4        5  Los Angeles
5        6      Seattle


In [ ]:
pd_host=(pd_matches.assign(HOST=np.where(pd_matches['HOST_GOALS']>pd_matches['GUEST_GOALS'],
                                         3,
                                         np.where(pd_matches['HOST_GOALS']<pd_matches['GUEST_GOALS'],
                                                  0,
                                                  1
                                            )
                                   )
                    )[['HOST_TEAM','HOST']]
                   .rename(columns={'HOST_TEAM':'TEAM_ID'})
)
print(f'scores achieved by host teams:\n{pd_host}')

scores achieved by host teams:
   TEAM_ID  HOST
0        1     3
1        2     0
2        3     3
3        4     1
4        5     3
5        6     0


In [ ]:
pd_guest=(pd_matches.assign(GUEST=np.where(pd_matches['HOST_GOALS']>pd_matches['GUEST_GOALS'],
                                           0,
                                           np.where(pd_matches['HOST_GOALS']<pd_matches['GUEST_GOALS'],
                                                    3,
                                                    1
                                              )
                                     )
                    )[['GUEST_TEAM','GUEST']]
                   .rename(columns={'GUEST_TEAM':'TEAM_ID'})
)
print(f'scores achieved by guest teams:\n{pd_guest}')

scores achieved by guest teams:
   TEAM_ID  GUEST
0        2      0
1        3      3
2        4      0
3        5      1
4        6      0
5        1      3


In [ ]:
pd_scores=(pd_teams.merge(pd_host.merge(pd_guest,
                                        on='TEAM_ID',
                                        how='inner'
                                  ),
                          on='TEAM_ID',
                          how='left'
                    )
)
query=(pd_scores.assign(SCORE=pd_scores['HOST']+pd_scores['GUEST'])
                .sort_values(by=['SCORE','TEAM_NAME'],
                             ascending=[False,True]
                 )
)[['TEAM_NAME','SCORE']]
print(f'total scores achieved by each team:\n{query}')

total scores achieved by each team:
     TEAM_NAME  SCORE
2      Chicago      6
0     New York      6
4  Los Angeles      4
3      Toronto      1
1      Atlanta      0
5      Seattle      0


In [ ]:
teams=pl.scan_csv(url2,
                  separator="\t",
                  has_header=True,
                  infer_schema_length=1000,
                  ignore_errors=False
)
matches=pl.scan_csv(url1,
                    separator="\t",
                    has_header=True,
                    infer_schema_length=1000,
                    ignore_errors=False
)

In [ ]:
sample1=teams.head(5)
print(f'----POLARS----\nteams table (SAMPLE):\n{sample1.collect()}')

----POLARS----
teams table (SAMPLE):
shape: (5, 2)
┌─────────┬─────────────┐
│ TEAM_ID ┆ TEAM_NAME   │
│ ---     ┆ ---         │
│ i64     ┆ str         │
╞═════════╪═════════════╡
│ 1       ┆ New York    │
│ 2       ┆ Atlanta     │
│ 3       ┆ Chicago     │
│ 4       ┆ Toronto     │
│ 5       ┆ Los Angeles │
└─────────┴─────────────┘


In [ ]:
sample2=matches.head(5)
print(f'matches table (SAMPLE):\n{sample2.collect()}')

matches table (SAMPLE):
shape: (5, 5)
┌──────────┬───────────┬────────────┬────────────┬─────────────┐
│ MATCH_ID ┆ HOST_TEAM ┆ GUEST_TEAM ┆ HOST_GOALS ┆ GUEST_GOALS │
│ ---      ┆ ---       ┆ ---        ┆ ---        ┆ ---         │
│ i64      ┆ i64       ┆ i64        ┆ i64        ┆ i64         │
╞══════════╪═══════════╪════════════╪════════════╪═════════════╡
│ 1        ┆ 1         ┆ 2          ┆ 3          ┆ 0           │
│ 2        ┆ 2         ┆ 3          ┆ 2          ┆ 4           │
│ 3        ┆ 3         ┆ 4          ┆ 4          ┆ 3           │
│ 4        ┆ 4         ┆ 5          ┆ 1          ┆ 1           │
│ 5        ┆ 5         ┆ 6          ┆ 2          ┆ 1           │
└──────────┴───────────┴────────────┴────────────┴─────────────┘


In [ ]:
host=(sample2.select(pl.col('HOST_TEAM')
                      .alias('TEAM_ID'),
                     pl.col('HOST_GOALS')
                       .alias('H'),
                     pl.col('GUEST_GOALS')
                       .alias('G')
              )
             .with_columns(SCORE_H=pl.when(pl.col('H')>pl.col('G'))
                                     .then(3)
                                     .otherwise(pl.when(pl.col('H')<pl.col('G'))
                                                  .then(0)
                                                  .otherwise(1)
                                      )
              )
)
print(f'scores achieved by host teams:\n{host.collect()}')

scores achieved by host teams:
shape: (5, 4)
┌─────────┬─────┬─────┬─────────┐
│ TEAM_ID ┆ H   ┆ G   ┆ SCORE_H │
│ ---     ┆ --- ┆ --- ┆ ---     │
│ i64     ┆ i64 ┆ i64 ┆ i32     │
╞═════════╪═════╪═════╪═════════╡
│ 1       ┆ 3   ┆ 0   ┆ 3       │
│ 2       ┆ 2   ┆ 4   ┆ 0       │
│ 3       ┆ 4   ┆ 3   ┆ 3       │
│ 4       ┆ 1   ┆ 1   ┆ 1       │
│ 5       ┆ 2   ┆ 1   ┆ 3       │
└─────────┴─────┴─────┴─────────┘


In [ ]:
guest=(sample2.select(pl.col('GUEST_TEAM')
                        .alias('TEAM_ID'),
                      pl.col('HOST_GOALS')
                        .alias('H'),
                      pl.col('GUEST_GOALS')
                        .alias('G')
               )
              .with_columns(SCORE_G=pl.when(pl.col('H')>pl.col('G'))
                                      .then(0)
                                      .otherwise(pl.when(pl.col('H')<pl.col('G'))
                                                   .then(3)
                                                   .otherwise(1)
                                       )
               )
)
print(f'scores achieved by guest teams:\n{guest.collect()}')

scores achieved by guest teams:
shape: (5, 4)
┌─────────┬─────┬─────┬─────────┐
│ TEAM_ID ┆ H   ┆ G   ┆ SCORE_G │
│ ---     ┆ --- ┆ --- ┆ ---     │
│ i64     ┆ i64 ┆ i64 ┆ i32     │
╞═════════╪═════╪═════╪═════════╡
│ 2       ┆ 3   ┆ 0   ┆ 0       │
│ 3       ┆ 2   ┆ 4   ┆ 3       │
│ 4       ┆ 4   ┆ 3   ┆ 0       │
│ 5       ┆ 1   ┆ 1   ┆ 1       │
│ 6       ┆ 2   ┆ 1   ┆ 0       │
└─────────┴─────┴─────┴─────────┘


In [ ]:
scores=(matches.select(pl.col('HOST_TEAM')
                         .alias('TEAM_ID'),
                       pl.col('HOST_GOALS')
                         .alias('H'),
                       pl.col('GUEST_GOALS')
                         .alias('G')
                )
               .with_columns(SCORE_H=pl.when(pl.col('H')>pl.col('G'))
                                       .then(3)
                                       .otherwise(pl.when(pl.col('H')<pl.col('G'))
                                                    .then(0)
                                                    .otherwise(1)
                                        )
                )
               .select(pl.col('TEAM_ID'),
                       pl.col('SCORE_H')
                )
               .join(matches.select(pl.col('GUEST_TEAM')
                                      .alias('TEAM_ID'),
                                    pl.col('HOST_GOALS')
                                      .alias('H'),
                                    pl.col('GUEST_GOALS')
                                      .alias('G')
                             )
                            .with_columns(SCORE_G=pl.when(pl.col('H')>pl.col('G'))
                                                    .then(0)
                                                    .otherwise(pl.when(pl.col('H')<pl.col('G'))
                                                                 .then(3)
                                                                 .otherwise(1)
                                                     )
                             )
                            .select(pl.col('TEAM_ID'),
                                    pl.col('SCORE_G')
                             ),
                      on='TEAM_ID',
                      how='full',
                      coalesce=True
                )
)
teams_rank=(teams.join(scores,
                       on='TEAM_ID',
                       how='left'
                  )
                 .with_columns(SCORE=pl.col('SCORE_H')+pl.col('SCORE_G')
                  )
                 .select(pl.col('TEAM_NAME'),
                         pl.col('SCORE')
                  )
                 .sort(by=['SCORE','TEAM_NAME'],
                       descending=[True,False]
                  )
)
print(f'scores achieved by each teams:\n{teams_rank.collect()}')

scores achieved by each teams:
shape: (6, 2)
┌─────────────┬───────┐
│ TEAM_NAME   ┆ SCORE │
│ ---         ┆ ---   │
│ str         ┆ i32   │
╞═════════════╪═══════╡
│ Chicago     ┆ 6     │
│ New York    ┆ 6     │
│ Los Angeles ┆ 4     │
│ Toronto     ┆ 1     │
│ Atlanta     ┆ 0     │
│ Seattle     ┆ 0     │
└─────────────┴───────┘
